In [1]:
using ImageMagick
using Knet
using JLD
using Statistics
using Random
using Images
include(Knet.dir("data", "mnist.jl"))
xtrain, ytrain, xtest, ytest = mnist();

┌ Info: Loading MNIST...
└ @ Main /home/ahnaf/.julia/packages/Knet/05UDD/data/mnist.jl:33


In [2]:
# Global Parameters
BANDWIDTH = 8
NUM_PATCHES = 4
GLIMPSE_COUNT = 2 #Number of glimpses to be employed
NUM_CLASSES = 10
SCALE = 2
HSIZE = 256
GSIZE = 256;
BATCHSIZE = 20
FILENAME = "RAM_fin_ref.jld2"

"RAM_fin.jld2"

In [3]:
# Generic Functions
# Normalize dataset and return mean and SD
function Normalize(dataset)
    mean_dataset = Float32(mean(dataset))
    sd_dataset = Float32(std(dataset))
    dataset_norm = (dataset .- mean_dataset) ./ sd_dataset
    return mean_dataset, sd_dataset, dataset_norm
end


# Function for normalizing the locations
# Batch location dimensions = 2 x Batchsize
function antinorm(dim, loc)
    #Conver Normalized locations range[-1,1] into X, Y coordinates
    anloc = []
    meanx::Int16 = floor(dim/2);
    meany::Int16 = floor(dim/2);
    
    locx = Array{Int16}(ceil.((loc[1,:] .* dim) ./ 2 .+ meanx))  
    locy = Array{Int16}(ceil.((loc[2,:] .* dim) ./ 2 .+ meany))
    for i = 1:BATCHSIZE
        temp = reshape([locx[i], locy[i]], (2,1))
        push!(anloc, temp)
    end
    return (cat(anloc..., dims = 2))
end

# # AntiNorm test
# test = KnetArray{Float32}(rand(2,BATCHSIZE))
# println(size(test))
# test = antinorm(28, test)

antinorm (generic function with 1 method)

In [4]:
MEANX, SDX, xtrain = Normalize(xtrain);
~, ~, xtest = Normalize(xtest);

dtrn = minibatch(xtrain, ytrain, BATCHSIZE, xtype=KnetArray{Float32});
dtst = minibatch(xtest, ytest, BATCHSIZE, xtype=KnetArray{Float32});

xd, yd = first(dtrn);

# Generic Elements

In [5]:
#################################################################################################
struct FC
    w
    b
    act
end

(f::FC)(x) =f.act.(f.w * mat(x) .+ f.b)
FC(outsize::Int, insize::Int, act = relu) = FC(param(outsize, insize, atype = KnetArray{Float32}), 
    param0(outsize, atype = KnetArray{Float32}), act);


# # Layer Test
# F1 = FC(100, 784)
# params(FC1)
# testFC = FC1(xd)
#################################################################################################

struct Linear
    w
    b
end

(l::Linear)(x) = l.w * mat(x) .+ l.b

function Linear(outsize::Int, insize::Int)
    w = param(outsize, insize, atype=KnetArray{Float32})
    b = param0(outsize, atype = KnetArray{Float32})
    return Linear(w,b)
end

##################################################################################################

# # Layer Test
# L1 = Linear(100, 784)
# params(L1)
# testLinear = L1(xd)

Linear

## Core Network
Takes the output of Glimpse Network(gt -> gc) and previous hidden state(h(t-1)-> hp). Outputs current Hidden State (ht -> hc).
Runs num_glimpses times and records the hidden state at the end as output. 
Unit Defintion = Rect (Linear(h(t-1)) + Linear(gt))

In [6]:
struct CoreNet
    FCgc   # Linear operator for g current  : gt
    FChp   # Linear operator for h previous : ht-1
end

function CoreNet(gsize::Int, hsize::Int)
    return CoreNet(Linear(gsize, hsize), Linear(hsize, hsize))
end

function (c::CoreNet)(gt,hp)
    return relu.(c.FCgc(gt) + c.FChp(hp))
end

# CoreNet Test
# CN = CoreNet(hsize, hsize)
# sample = KnetArray{Float32}(rand(256,10))
# sample = reshape(sample , (256, 10))
# CN_test = CN(sample, sample)

# Glimpse Network
## Glimpse Network Constructor

In [7]:
mutable struct GpsNet
    FC1
    FC2
    FC3
    FC4
end

### Glimpse Network: Initializer & Method

In [8]:
# Glimpse Network default constructor: Definition of vairous cogs of the Glimpse network
function GpsNet(bandwidth::Int, scale::Int, num_patches::Int)
    FC1 = FC(128, bandwidth*bandwidth*num_patches, identity)
    FC2 = FC(128, 2, identity)
    FC3 = Linear(256, 128)  # The outsize may be 256
    FC4 = Linear(256, 128) # The outsize may be 256
    return GpsNet(FC1, FC2, FC3, FC4)
end
###########################################################################################

## Glimpse Network Deployment

# Deploying glimpse Network
# Input is the batch of images
# Take glimpses & locations. Pass them to FC layers 
function (g::GpsNet)(x_batch, loc)
    phi   = sensor(x_batch, loc)
    phi_1 = g.FC1(phi)
    loc_1 = g.FC2(loc)
    phi_2 = g.FC3(phi_1)
    loc_2 = g.FC4(loc_1)
    return relu.(phi_2 + loc_2) # The output here is gt to be fed to CoreNet
end

# # GpsNet Test
# test2 = GpsNet(bandwidth, scale, num_patches)
# params(test2.FC2)
# res_test2 = test2(xd, loc)

### Sensor Network

In [9]:
# Uses imresize for equalizing the return dimensions
function sensor(x_batch, loc)
    l,w,c,b = size(x_batch)
    coord = antinorm(l, loc)
    glimpse_array = []
    padsize::Int = 60
    width::Int = floor(BANDWIDTH / 2)  
    batch = Array(mat(x_batch))
    batch = reshape(batch, (l,w,b))
    
    for i = 1:BATCHSIZE
        img = batch[:,:,i]
        pad_img = padarray(img, Fill(0, (padsize, padsize)))
        temp = []
        for j = 1: NUM_PATCHES
            xlim1 = coord[1,i] - (width* (SCALE ^ (j-1)))
            xlim2 = coord[1,i]-1 + (width * (SCALE ^(j-1)))
            ylim1 = coord[2,i] - (width* (SCALE ^ (j-1)))
            ylim2 = coord[2,i]-1 + (width * (SCALE ^(j-1)))

            glimpse = pad_img[xlim1:xlim2, ylim1:ylim2]
            glimpse = imresize(glimpse, (BANDWIDTH, BANDWIDTH))
            push!(temp, glimpse)
        end
        temp = vcat(vcat(temp...)...)
        push!(glimpse_array, vcat(temp))
    end
    return KnetArray{Float32}(mat(cat(glimpse_array..., dims=2)))
end

## Function test
# coord = KnetArray{Float32}(zeros(2, BATCHSIZE))
# res = sensor(xd, coord)

sensor (generic function with 1 method)

## Location Network

In [10]:
mutable struct LocNet
    sdx
    Linl
end

# Function should take input size as 256 and output size as 2 (size of loc coordinates)
function LocNet(outsize::Int, insize::Int, sdx::Float32)
    Linloc = Linear(outsize, insize)
    return LocNet(sdx, Linloc)
end

function (l::LocNet)(ht)
    mean_loc = tanh.(l.Linl(ht))
    
    prtb = KnetArray{Float32}(gaussian((2,BATCHSIZE); mean = 0, std = SDX))
    loc_new = mean_loc + prtb 
    return mean_loc, tanh.(loc_new)
end

# # TLocation Network Test
# Loc1 = LocNet(2, 256, SDX)
# sample = KnetArray{Float32}(rand(256));
# sample = reshape(sample, (256, 1));
# res, lt = Loc1(sample)

## RAM Model
### Staggered RAM
The Reference RAM deployment has been staggered.
Deploy one RAM function once to generate the hidden state, next location and action.
Deploy this one-time active RAM recursively equal to number of glimpses. Record all intermediate states
but base decision on output of the final one of the loop.

In [11]:
# RAM Initializer
# ram = RAM(SDX, GLIMPSE_COUNT, BANDWIDTH, NUM_PATCHES, NUM_CLASSES, SCALE, GSIZE, HSIZE);
mutable struct RAM
    sdx
    num_glimpses
    gpsnet::GpsNet
    corenet::CoreNet
    locnet::LocNet
    output_layer
    locations
    basenet
    hsize
end

function RAM(sdx, glimpse_count, bandwidth, num_patches, num_classes, scale, gsize, hsize)
#     sdx = SDX
#     num_glimpses = glimpse_count
    gpsnet = GpsNet(bandwidth, scale, num_patches)
    corenet = CoreNet(gsize, hsize)
    locnet = LocNet(2, hsize, sdx)
    output_layer = Linear(num_classes, hsize) # Action Network
    locations = []
    basenet = FC(1, hsize)
    return RAM(sdx, glimpse_count, gpsnet, corenet, locnet, output_layer, locations, basenet, hsize)
end

##########################################################################################

# Outer RAM instance. Calls inner instance glimpse_count times
function (r::RAM)(x_batch)
    # Init States
    hc = KnetArray{Float32}(reshape(zeros(r.hsize, BATCHSIZE), (r.hsize, BATCHSIZE)))
    loc = KnetArray{Float32}(zeros(2,BATCHSIZE))
    
    log_ps, baseresults = [], []
    for i = 1:r.num_glimpses
        hc, loc, policy, baseres = r(x_batch, loc, hc)
        push!(log_ps, policy)       
        push!(baseresults, baseres)      
    end
    log_ps, baseresults = vcat(log_ps...), vcat(baseresults...)    
    scores = logsoftmax(r.output_layer(mat(hc)))
    return scores, log_ps, hc, baseresults
end

###########################################################################################

# Innermost RAM instance
function (r::RAM)(x_batch, loc, hp)
    gc = r.gpsnet(x_batch, loc)  # g-current: gt
    hc = r.corenet(gc,hp)       # h-current ; ht
    mean_loc, lnext = r.locnet(hc) # mean and l-next:lt to be used in the next time step
  
    #Gaussian PDF for REINFORCE
    policy = -(abs.(value(lnext) - mean_loc).^2) / 2*(r.sdx^2) .- log(r.sdx) .- log(sqrt(2 * 3.1415))
    policy = sum(policy, dims =1)
    base_res = r.basenet(hc)
    return hc, lnext, policy, base_res
end

In [12]:
# Loss Caluclator
function (ram::RAM)(x_batch, y)
    scores, log_ps, hc, baseresults = ram(x_batch) 
    ypred = vec(map(i->i[1], argmax(Array(value(scores)), dims=1)))
    r = ypred .== y; r = reshape(r, 1, :)
    R = zeros(Float32, size(baseresults)...); R[end,:] = r
    R = KnetArray{Float32}(R)
    R̂ = R .- value(baseresults)
   
    loss_action = nll(scores, y)
    loss_baseline = sum(abs2, baseresults .- R) / length(baseresults)
    loss_reinforce = mean(sum(-log_ps .* R̂, dims=1))
    return loss_action, loss_baseline, loss_reinforce, sum(R), length(R)
end

In [13]:
function validate(ram::RAM, data)
    losses = zeros(3)
    ncorrect = ninstances = 0
    for (x,y) in data
        ret = ram(x,y)
        for i = 1:3; losses[i] += ret[i]; end
        ncorrect += ret[4]
        ninstances += ret[5]
    end
    losses = losses / length(data)
    losses = [sum(losses), losses...]
    return losses, ncorrect / ninstances
end

validate (generic function with 1 method)

### RAM Initializer

In [14]:
function initopt!(model, optimizer="Adam(lr=0.0003, gclip=0.0)")
    for par in params(model)
        par.opt = eval(Meta.parse(optimizer))
    end
end

ram = RAM(SDX, GLIMPSE_COUNT, BANDWIDTH, NUM_PATCHES, NUM_CLASSES, SCALE, GSIZE, HSIZE);
record = params(ram);
initopt!(ram)

In [16]:
ram = Knet.load("RAM_fin.jld2", "RAM");

In [ ]:
epochs = 1000
history = []
best_acc = 0.0
loss(x, yref) = sum(ram(x, yref)[1:3])
gradients = []
for epoch = 1:epochs
    losses = []
    for (x, y) in dtrn
        lss = @diff loss(x, y)
        push!(losses, value(lss))
        for par in params(ram)
            g = grad(lss, par)
            update!(value(par), g, par.opt)
        end
    end
    println("Loss for #epoch : ", mean(losses))
    trn_losses, trn_acc = validate(ram, dtrn);
    tst_losses, tst_acc = validate(ram, dtst);
    println(
        "epoch=$(1+length(history)) ",
        "trnloss=$(trn_losses), trnacc=$trn_acc, ",
        "tstloss=$(tst_losses), tstacc=$tst_acc")
    push!(history, ([trn_losses..., trn_acc, tst_losses..., tst_acc]));
    
    if tst_acc > best_acc
            best_acc = tst_acc
            Knet.save(FILENAME, "RAM", ram)
            Knet.@save "History_fin.jld2" history
    end
        
end

Loss for #epoch : 0.17426784
epoch=1 trnloss=[0.145994, 0.119664, 0.0152582, 0.011072], trnacc=0.484225, tstloss=[0.271182, 0.231468, 0.0214549, 0.0182594], tstacc=0.4776
Loss for #epoch : 0.12563308


In [ ]:
Knet.save(FILENAME, "RAM", ram)
Knet.@save "History_ref.jld2" history

In [ ]:
ram = nothing
Knet.gc()